In [120]:
import wave
import json
from pydub import AudioSegment as AS
from vosk import Model, KaldiRecognizer, SetLogLevel
import Word as custom_Word

In [121]:
model_path = "model"

audio_filename = input("please enter relative path for the file you wish to analyze in .wav format")

model = Model(model_path)
wf = wave.open(audio_filename, "rb")
rec = KaldiRecognizer(model, wf.getframerate())
rec.SetWords(True)

results = []

while True:
    data = wf.readframes(4000)
    if len(data) == 0:
        break
    if rec.AcceptWaveform(data):
        part_result = json.loads(rec.Result())
        results.append(part_result)
part_result = json.loads(rec.FinalResult())
results.append(part_result)

In [122]:
list_of_Words = []
list_of_se = []

for sentence in results:
    if len(sentence) == 1:
        continue
    for obj in sentence['result']:
        w = custom_Word.Word(obj) 
        se = w.start_end()
        list_of_Words.append(w)
        list_of_se.append(se)

In [123]:
list_of_Words_string = []
for word in list_of_Words:    # print(word.to_string())
    list_of_Words_string.append(word.word_return())

output = []
for i in range(len(list_of_Words_string)):
    output_temp = [list_of_Words_string[i], list_of_se[i]]
    output.append(output_temp)

print(output)

[['alexa', '0.33, 1.08'], ['what', '1.44, 1.71'], ['is', '1.71, 1.89'], ['the', '1.89, 2.01'], ['weather', '2.01, 2.39'], ['on', '2.40, 2.61'], ['slanging', '2.61, 3.18'], ['alexa', '3.84, 4.53'], ['what', '4.98, 5.22'], ['is', '5.22, 5.43'], ["today's", '5.43, 6.03'], ['date', '6.03, 6.54'], ['alexa', '7.56, 8.22'], ['what', '8.28, 8.52'], ['time', '8.52, 8.85'], ['is', '8.85, 9.06'], ['it', '9.06, 9.21'], ['alexa', '9.87, 10.59'], ['when', '10.65, 10.92'], ['is', '10.92, 11.07'], ['thanksgiving', '11.07, 11.97'], ['collect', '12.90, 13.26'], ['snap', '13.26, 13.62'], ['how', '14.10, 14.25'], ['do', '14.25, 14.37'], ['you', '14.37, 14.55'], ['bake', '14.58, 14.91'], ['chocolate', '14.91, 15.45'], ['chip', '15.48, 15.75'], ['cookies', '15.75, 16.38'], ['alexa', '17.10, 17.79'], ['what', '18.51, 18.75'], ['is', '18.75, 18.96'], ['two', '18.96, 19.26'], ['times', '19.26, 19.80'], ['four', '19.80, 20.16'], ['sure', '20.16, 20.34'], ['alexa', '21.36, 22.02'], ['how', '22.59, 22.77'], ['man

In [124]:
alexa_list = []
for i in output:
    if i[0] == "alexa":
        split_alexa = i[1].split(",")
        i = [i[0], split_alexa[0], split_alexa[1]]
        alexa_list.append(i)


def clip_file(alexa_begin, alexa_end):
    part = AS.from_file(audio_filename)
    t1 = float(alexa_begin[1]) * 1000
    t2 = float(alexa_end[1]) * 1000
    clip = part[t1:t2]
    return clip


clipped_alexa_files = []
z = 0
while z+1 != len(alexa_list):
    clipped_alexa_files.append(clip_file(alexa_list[z], alexa_list[z+1]))
    z += 1

def _19(alexa_begin):
    part = AS.from_file(audio_filename)
    t1 = float(alexa_begin[1]) * 1000
    clip = part[t1:]
    return clip

last_command = _19(alexa_list[-1])
clipped_alexa_files.append(last_command)

In [125]:
audio_filename = audio_filename.split("\\")

In [126]:
count = 0
while count != len(clipped_alexa_files):
        name = ("command" + str(count+1) + audio_filename[0] + audio_filename[1])
        clipped_alexa_files[count].export(name, format="wav")
        count += 1